<a href="https://colab.research.google.com/github/Eleftheria-99/gpt-finetuned-greek-mitos-dataset/blob/main/GPT2_FINETUNED_GREEK_MITOS_DATASET_EL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate peft bitsandbytes==0.41.2.post2 trl transformers==4.35.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.1/244.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 13.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (41

In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from google.colab import drive
import pandas as pd
from datasets import Dataset, DatasetDict


# Model from Hugging Face hub
base_model = "lighteternal/gpt2-finetuned-greek"

#Finetuned model
new_model = "gpt2-greek"

# Instruction dataset
drive.mount('/content/drive')

# Create a sample Pandas DataFrame
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DATASET/MITOS-EL-QAS.csv', delimiter =',')

# Convert Pandas DataFrame to Hugging Face Dataset and split to train and test
datasets = Dataset.from_pandas(df).train_test_split(test_size=0.2)

dataset = datasets["train"]
eval_dataset = datasets["test"]

# Creation of 4-bit quantization with NF4 type configuration using BitsAndBytes
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Loading model using 4-bit precision with the compute dtype "float16" from Hugging Face for faster training
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Loading tokenizer from Hugginface and set padding_side to “right” to fix the issue with fp16
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# PEFT parameters
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Training parameters
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Model fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

# Save the model adopter and tokenizers
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

logging.set_verbosity(logging.CRITICAL)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/755 [00:00<?, ? examples/s]

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
prompt = "Ποιες είναι οι προϋποθέσεις για την έκδοση της άδειας μεταφορέα ζώντων ζώων τύπου ΙΙ;"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Ποιες είναι οι προϋποθέσεις για την έκδοση της άδειας μεταφορέα ζώντων ζώων τύπου ΙΙ; Η μελέτη, ο έλεγχος και η αξιολόγηση της καταλληλότητας των εγκαταστάσεων αυτών, στην περίπτωση που αφορούν τις σύγχρονες εγκαταστάσεις του ζωικού κεφαλαίου θα πραγματοποιηθεί βάσει ειδικής μελέτης (ή σχετικής απόφασης από τη Διεύθυνση Αγροτικής Οικονομίας). Οι απόψεις αυτές διατυπώνονται κατά κανόνα σε μία μόνο πρόταση. Στην πραγματικότητα το περιεχόμενο αυτής μπορεί να αποτελέσει αντικείμενο διαλόγου με τους εμπλεκόμενους φορείς:Ο βουλευτής Πιερίας μαζί μ’ άλλους τέσσερις βουλευτές -τον πρώην πρωθυπουργό Γιώργο Παπανδρέου- συμμετείχε στη συνεδρίαση όπου παρουσία όλων ήταν συγκεντρωμένος όλη η πολιτική ηγεσία στο ξενοδοχείο Dakar προκειμένου «να μάθει» τι συνέβαινε στα γραφεία αλλά κυρίως στους χώρους εργασίας όπως ακριβώς έχουν καταγραφεί στις συνομιλίες…Κατά τα άλλα φαίνεται ότι δεν πρέπει κανείς σου ούτε εσύ,ουτε κάποιος άλλος σ αυτόν τον τόπο ή ένα κόμμα.Αυτή λοιπόν εδώ έχει μεγαλύτερη ανάγκη απο

In [ ]:
prompt = "Βήματα της διαδικασίας έκδοσης άδειας διακίνησης αιγών και προβάτων"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Βήματα της διαδικασίας έκδοσης άδειας διακίνησης αιγών και προβάτων από την Κ.Α του ΟΠΕΚΕΠΕ (όπως έχει διαμορφωθεί κατά το χρόνο της έκδοσης αυτής) για τα οποία δεν έχουν εκδοθεί οι σχετικές άδειες εμπορίας, καθώς επίσης θα γίνουν ανάλογες επενδύσεις ανά νομό στους παραγωγούς-παραγωγούς. Οι επιχειρήσεις που έλαβαν άδεια κυκλοφορίας υποχρεούνται να μην επιτρέπουν στην επιχείρηση τη μεταφορά των προϊόντων τους ή άλλων αντικειμένων εκτός της εγκαταστάσεις παραγωγής (πτήσεις κλπ). Τα δικαιώματα αυτά ισχύουν ακόμη στην περίπτωση όπου ο παραγωγός – πωλητής οφείλει σύμφωνα με τις διατάξεις ν’ αρνηθεί οποιαδήποτε αλλαγή σε οποιοδήποτε προϊόν παρά τον περιορισμό ότι μπορεί στο μέλλον να αποκτήσει νέο εμπορικό σήμα εντός 2 ετών μετά όμως αυτός πλέον φέρει σχετική ευθύνη λόγω μη παροχής ενημέρωσης σχετικά στα προϊόντα αλλά ούτε είναι δυνατόν η πώληση των εν τω μεταξύ μεταποιητικών προϊόντων προς τρίτους εφόσον γίνει υπό αυτές συνθήκες στις οποίες υπάρχει κίνδυνος να προκαλέσουν ζημιές στον τελικό

In [ ]:
prompt = "Ποια είναι τα βήματα της διαδικασίας αλλαγής στοιχείων επικοινωνίας αλλοδαπών;"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Ποια είναι τα βήματα της διαδικασίας αλλαγής στοιχείων επικοινωνίας αλλοδαπών;«Η ιδέα που μου ήρθε με το Cabo Seminar, ήταν να δώσω σε όλους τους ανθρώπους τη δυνατότητα ενός παιχνιδιού. Ήταν ένα από αυτά και ήθελα πάρα πολύ ν' αποκτήσουν κάποιες γνώσεις πάνω στο θέμα αυτό», είχε δηλώσει ο 22χρονος πρώην παίκτης στην «One City Daily» σχετικά για την εμπειρία του ως προπονητής των Bucks Jets (2016/2017).Ο Δήμος Λέσβου διαθέτει σήμερα τουλάχιστον ένα ξενοδοχείο πέντε αστέρων στη Λέσβο καθώς επίσης και οκτώ ακόμη πολυτελείς βίλες όπου μπορείτε πραγματικά μόνο εσείς οι ίδιοι η οικογένεια σας! Αν δε θέλετε τις διακοπές ή αν δεν είστε υποχρεωμένοι θα θέλατε τις δύο πρώτες εβδομάδες κάθε μήνα μέχρι τον Σεπτέμβριο οπότε πρέπει πλέον είτε απλά μένετε σπίτι - χωρίς χρήματα- αφού αυτή πραγματοποιείται 2-3 μέρες νωρίτερα αλλά πριν πάτε εκεί ίσως έχει συμβεί κάτι σημαντικό στον χώρο αναψυχής...Οι υποψήφιοι έχουν δικαίωμα πρόσβασης στις ηλεκτρονικές υπηρεσίες της Γενικής Γραμματείας Πληροφοριακών Συ

In [ ]:
prompt = "Τι αφορα η διαδικασία έκδοσης δελτίου ανεργίας;"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Τι αφορα η διαδικασία έκδοσης δελτίου ανεργίας; Απο την πλευρά μου έχω δηλώσει οτι θα το διορθώσω. Να 'μαι καλά... και να δώσω τις οδηγίες για τον τρόπο που πρέπει κάποιος/α άνεργος, με οποιοδήποτε κόστος! Δεν είναι θέμα των εταιρειών ή της διοίκησης αλλά του καθενός από εμάς τους ίδιους(και όχι μόνον )που βιώνουμε καθημερινά στην καθημερινότητά μας τέτοιες καταστάσεις. Αν είσαι απο αυτούς (αν δεν είμαι εγώ) υπάρχει περίπτωση απλά κάποια στιγμή μέσα στον μήνα οι υπηρεσίες σου στη χώρα αυτή,να κλείσουν ;Η συμμετοχή στις εξετάσεις μπορεί να ξεκινήσει είτε απευθείας στις 9:00 πμ-17 μμ όπου όλοι μαζί κάνουμε μια μικρή στάση εργασίας -η οποία έχει διάρκεια 4 ώρες κάθε μέρα-, πριν τη λήξη μιας σειράς εξετάσεων μέχρι 24ωρου – οπότε δίνουμε τα αποτελέσματα στα μέλη ανάλογα πάντα απ’το χρόνο αναμονής καθόλη τη βδομάδα…Για παράδειγμα αν σας κάνει εντύπωση ο χρόνος προσέλευσης στο κέντρο εξυπηρέτησης πελατών ώστε αυτός αυτόματα δε διαθέτει πρόσβαση προς κάποιο κατάστημα τότε μάλλον κάτι τέτοιο εν

In [ ]:
prompt = "Τι αφορά η διαδικασία Δήλωση Ε9 / ΕΝΦΙΑΗ;"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Τι αφορά η διαδικασία Δήλωση Ε9 / ΕΝΦΙΑΗ; Με ποιο λόγο, οι «αλεπούδες» δεν μπορούν να πληρώσουν το φόρο εισοδήματος. Τα νοικοκυριά θα πρέπει πλέον με την υποβολή των δηλώσεων ΦΑΠ και ΦΠΑ στο Taxisnet ή σε άλλα εφοριακά συστήματα να είναι υποχρεωτικά υπό τον έλεγχο του υπουργείου Οικονομικών – που ουσιαστικά έχει επιφορτιστεί ήδη - για τα πρόστιμα της φοροδιαφυγής ώστε αυτοί στην αρχή κι ύστερα όλοι εμείς ως φορολογούμενοι…Τα αποτελέσματα έδειξαν ότι όσοι είχαν τουλάχιστον τρεις από τις πέντε γυναίκες (πάνω των 15 ετών) εμφάνισαν μεγαλύτερη μείωση κατά τη διάρκεια αυτών μετά τους 18 μήνες άσκησης χωρίς διακοπή καθόλη τη ζωή λόγω ορμονικών αλλαγών όπως αύξηση σωματικού βάρους εξαιτίας στρες κ άγραφου νόμου στη ρύθμιση περί καρκίνου μαστού καθώς επίσης αυξημένη όρεξη κα ινομυαλγία.Ο Πρόεδρος είπε ακόμη πως ο χρόνος αναμονής μέχρι τελικά βρεθεί λύση εξαρτάται από άλλους παράγοντες: "Είμαι σίγουρος ότι εάν βρούμε κάποια λύση μπορεί είτε αυτό προκύψει μέσα από συνεννόηση μεταξύ μας (πχ Τουρκ

In [ ]:
!pip install evaluate rouge_score np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=211df47d501753b2f26079fc12ec9a04ee6c24ab701d958f0ec34ac8a79baf44
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
  Created wheel for np: filename=np-1.0.2-py3-none-any.whl size=13657 sha256=2689c2934fa9f31d41d78767bcf7f2a184c8e2f474af355c37418da0409efe66
  Stored in directory: /root/.cache/pip/wheels/f9/e1/79/c4580a03022dece233bc146286fc9082a8885ee1a7c8d8b980
Successfully built rouge_score np


In [ ]:
# Evaluation of the model -- BLEU
import evaluate
bleu = evaluate.load('bleu')

references = eval_dataset["text"]
predictions = []

for data in eval_dataset:
  # print("question - answer:", data["text"])
  prompt = data["text"]

  if(prompt != ""):
    prompt = prompt.split("\n")
    if(len(prompt) > 0):
      prompt = prompt[0]
      pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
      result = pipe(f"{prompt}")
      result = result[0]['generated_text']
      predictions.append(result)

# print("references: ", references)
# print("predictions: ", predictions)

bleu_results = bleu.compute(predictions=predictions, references=references, max_order = 2)
print("bleu_results")
print(bleu_results)


bleu_results
{'bleu': 0.033355613339466156, 'precisions': [0.04831823388524419, 0.023026440575091994], 'brevity_penalty': 1.0, 'length_ratio': 11.430905746131371, 'translation_length': 105633, 'reference_length': 9241}


In [ ]:
# Evaluation of the model -- ROUGE - of the initial model
import evaluate
import numpy as np

rouge = evaluate.load('rouge')

references = eval_dataset["text"]
predictions = []

# GPT2 FINETUNED MODEL AS INITIAL MODEL

model = "lighteternal/gpt2-finetuned-greek"

generator = pipeline(
          'text-generation',
          #device=0,
          # model=f'{model}',
          # tokenizer=f'{tokenizer}',
          model=model,
          tokenizer=tokenizer,
          max_length=700
      )

for data in eval_dataset:
  prompt = data["text"]
  if(prompt != ""):
    prompt = prompt.split("\n")
    if(len(prompt) > 0):
      prompt = prompt[0]

      result = "\
      ".join([x.get("generated_text") for x in generator(
          prompt,
          max_length=len(prompt.split(" "))+15,
          do_sample=True,
          top_k=50,
          repetition_penalty = 1.2,
          add_special_tokens=False,
          num_return_sequences=5,
          temperature=0.95,
          top_p=0.95)])
      predictions.append(result)

original_model_results = rouge.compute(
    predictions=predictions,
    references=references,
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 28, but `max_length` is set to 28. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 37, but `max_length` is set to 33. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 36, but `max_length` is set to 33. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


ORIGINAL MODEL:
{'rouge1': 0.0339240859082129, 'rouge2': 0.01055547650159179, 'rougeL': 0.033495832503769016, 'rougeLsum': 0.03348432872242396}


In [ ]:
# Evaluation of the model -- ROUGE - of the finetuned model
import evaluate

rouge = evaluate.load('rouge')

references = eval_dataset["text"]
predictions = []

for data in eval_dataset:
  prompt = data["text"]

  if(prompt != ""):
    prompt = prompt.split("\n")
    if(len(prompt) > 0):
      prompt = prompt[0]
      pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
      result = pipe(f"{prompt}")
      result = result[0]['generated_text']
      predictions.append(result)

peft_model_results = rouge.compute(
    predictions=predictions,
    references=references,
    use_aggregator=True,
    use_stemmer=True,
)

print('PEFT MODEL:')
print(peft_model_results)

PEFT MODEL:
{'rouge1': 0.045211011879953536, 'rouge2': 0.004404654924633031, 'rougeL': 0.03276432806886516, 'rougeLsum': 0.0439029235765338}


In [ ]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 1.13%
rouge2: -0.62%
rougeL: -0.07%
rougeLsum: 1.04%
